# **Extract GridMET Data**

By Bridget Bittmann

Date created: March 28, 2022

Date editted: 

Purpose: This script extracts GridMET data from Google Earth Engine using the Earth Engine API in Python.

In [1]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()  

geemap package not installed. Installing ...
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=Id4jA4yy0d6mVSaIMvyiNFSneJloSwjGN4lO2KXdJvY&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhiQAF3PeQHVch5p3vyoFBPyiotCg9FENwAdP4do9vDYEP07LrS_3Y

Successfully saved authorization token.


In [2]:
!pip install geopandas
import geopandas as gpd #import independent shapefile
import json #for metadata of shapefile
import os #for file paths
import numpy as np #for stats and arrays
import pandas as pd #for dataframes

     |████████████████████████████████| 1.0 MB 21.6 MB/s 
     |████████████████████████████████| 16.7 MB 455 kB/s 


In [3]:
#Connect to Google Drive if you want to export images
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Go to correct folder in Google Drive
%cd drive/MyDrive/spatial_colab/datasets/
%ls

/content/drive/MyDrive/spatial_colab/datasets
2021_clip/      climate_stats/         IrrMapper/    subset_test_shp/
Boise_CDL/      diversion_timeseries/  LBRB_shp/
CDL_reproject/  irrigation_companies/  lcmap_files/
CDL_subsets/    irrig_lbrb/            masked/


In [5]:

# Algorithm:
# 1. Import shapefile of chosing
# 3. Create a new feature with selected watershed
# 5. Union the watersheds (if necessary), export the watershed(s)
# 6. Get the CDL data
# 7. Clip to the area of the watershed
# 8. Export 

#--------------------------------#
#   Name           |    huc8     #
#--------------------------------#
#   Lower Boise    |  17050114   #
#--------------------------------#




In [6]:
## ----------------------------------- ## 
## 1. Import shapefile to clip dataset ##
## ----------------------------------- ## 

shp_file = 'subset_test_shp/subset.shp'
subset = emap.shp_to_ee(shp_file)

map=emap.Map(center=(43.6150, -116.2023),zoom=8)
map.addLayer(ee.Image().paint(subset, 0, 2), {}, 'POU')
map.addLayerControl()
map

In [7]:
## -------------------------- ## 
## 2. Import the GridMET data ##
## -------------------------- ## 

dataset = ee.ImageCollection("IDAHO_EPSCOR/GRIDMET").filterDate('1987-01-01', '2020-12-31')
dataset.first()
max_temp = dataset.select('tmmx') \
            .map(lambda image:image.clip(subset))

max_tempVis = {
    'min': 290.0,
    'max': 314.0,
    'palette': ['d8d8d8', '4addff', '5affa3', 'f2ff89', 'ff725c']

}

map = emap.Map(center=(43.6150, -116.2023),zoom=8)
map.addLayer(max_temp , max_tempVis,"Maximum Temperature")

map.addLayerControl()
map


In [8]:
## ------------------------------------ ##
## IMPORT THE DAYMET DATA FOR MAX TEMPS ##
## ------------------------------------ ##

years = np.arange(1985,2021)
mean_max = []
for i in range(len(years)):
  daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(years[i])+'-06-01'), (str(years[i])+'-8-31')) #get image collection
  mxtmp = daymet.select('tmax').map(lambda image: image.clip(subset)).mean() #select the bands to analyze
  mean_max.append(mxtmp) #calculate the mean across all pixels

means_max_temp = ee.ImageCollection(mean_max) #convert list of image to image collection for zonal stats command

maximumTemperatureVis = {
  'min': -40.0,
  'max': 30.0,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

Map = emap.Map(center=(43.6150, -116.2023),zoom=8)
Map.addLayer(means_max_temp, maximumTemperatureVis, 'tmax')
Map

In [9]:
## ---------------------------------------- ##
## IMPORT THE DAYMET DATA FOR PRECIPITATION ##
## ---------------------------------------- ##

years = np.arange(1985,2021)
sum_pr = []
for i in range(len(years)):
  daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(years[i])+'-01-01'), (str(years[i])+'-12-31')) #get image collection
  prcp = daymet.select('prcp').map(lambda image: image.clip(subset)).sum() #select the bands to analyze
  sum_pr.append(prcp) #calculate the mean across all pixels

sum_precip = ee.ImageCollection(sum_pr) #convert list of image to image collection for zonal stats command

maximumTemperatureVis = {
  'min': 0,
  'max': 544,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

Map = emap.Map(center=(43.6150, -116.2023),zoom=8)
Map.addLayer(means_max_temp, maximumTemperatureVis, 'tmax')
Map

In [10]:
## --------------------- ##
## CALCULATE ZONAL STATS ##
## --------------------- ##

# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM

out_stats = os.path.join('climate_stats/maxtemp_stats.csv')
emap.zonal_statistics(means_max_temp, subset, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('climate_stats/precip_stats.csv')
emap.zonal_statistics(sum_precip, subset, out_stats, statistics_type='MEAN', scale=1000)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/spatial_colab/datasets/climate_stats/maxtemp_stats.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/spatial_colab/datasets/climate_stats/precip_stats.csv


In [41]:
## ------------------------------------------- ##
## CREATE CLIMATE STAT FOR EACH POU AND EXPORT ##
## ------------------------------------------- ##

precip = pd.read_csv('climate_stats/precip_stats.csv')
max_temp = pd.read_csv('climate_stats/maxtemp_stats.csv')

names = precip['DIV_NAME_']

for i in range(len(names)):
  df = pd.DataFrame(years, columns=['Year'])
  df['DIV_NAME'] = names[0]
  df['Precip_mm'] = precip.iloc[0,0:36].values
  out_path = os.path.join('climate_stats/'+names[i]+'_climate.csv')
  df.to_csv(out_path)
